<a href="https://colab.research.google.com/github/guilherme-argentino/fiap-ia4devs-techchallenge-fase4/blob/main/Atividade4Corrigida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get update && apt-get install -y libgl1-mesa-glx
!pip install opencv-python matplotlib mediapipe

!pip install --upgrade numpy deepface fpdf


Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [69.9 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,378 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu 

In [6]:
import cv2
import os
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from google.colab.patches import cv2_imshow
from deepface import DeepFace
import mediapipe as mp
import numpy as np
import gdown
from collections import Counter
import time
import pandas as pd
from fpdf import FPDF

In [4]:
# Baixar o vídeo diretamente do Google Drive
VIDEO_ID = "1B5PbZdUDi-r7Ac7BK3a3WdNppfQqM_Ne"  # ID do arquivo no Google Drive
VIDEO_PATH = "/content/video.mp4"
gdown.download(f"https://drive.google.com/uc?id={VIDEO_ID}", VIDEO_PATH, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1B5PbZdUDi-r7Ac7BK3a3WdNppfQqM_Ne
To: /content/video.mp4
100%|██████████| 38.3M/38.3M [00:01<00:00, 22.5MB/s]


'/content/video.mp4'

In [8]:
def detect_faces_and_activities(video_path, output_path, duration_seconds=None):
    """Detecta rostos, emoções e atividades no vídeo."""

    # Modelos do MediaPipe para detecção de atividades
    mp_holistic = mp.solutions.holistic
    holistic = mp_holistic.Holistic()

    # Modelo de detecção de rosto
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Abrindo o vídeo
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Erro: Não foi possível abrir o vídeo.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Obtendo propriedades do vídeo
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Definir limite de quadros para teste rápido
    if duration_seconds:
        max_frames = min(total_frames, int(fps * duration_seconds))
    else:
        max_frames = total_frames

    # Inicializando armazenamento para resumo
    emotion_counter = Counter()
    activity_counter = Counter()
    anomaly_count = 0
    prev_landmarks = None  # Para comparação de movimento

    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    processed_frames = 0

    start_time = time.time() # Inicia a contagem do tempo

    print(f"Frames processados: {processed_frames}/{total_frames}")

    frame_count = 0  # Contador de quadros processados

    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Converter para escala de cinza (para detecção de rosto)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar rostos
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        emotion = None  # Inicializa emoção do frame

        # Processar cada rosto detectado
        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]  # Extrair a região do rosto

            try:
                # Analisar emoção usando DeepFace
                analysis = DeepFace.analyze(face, actions=['emotion'], enforce_detection=False)
                emotion = analysis[0]['dominant_emotion']  # Obter emoção dominante

                # Armazena a emoção detectada
                emotion_counter[emotion] += 1

                # Desenhar retângulo no rosto
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

                # Adicionar a emoção detectada no vídeo
                cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Erro na análise de emoção: {e}")

        # **Detecção de atividade humana (pose + mãos + rosto)**
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = holistic.process(frame_rgb)

        activity = "Desconhecida"

        if result.pose_landmarks:
            mp_drawing = mp.solutions.drawing_utils
            mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

            # Pegar marcos corporais
            landmarks = result.pose_landmarks.landmark
            left_shoulder = landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER].y
            left_knee = landmarks[mp_holistic.PoseLandmark.LEFT_KNEE].y

            # Determinar se a pessoa está em pé ou sentada
            if left_shoulder < left_knee:
                activity = "Em Pe"
            else:
                activity = "Sentado"

        if result.face_landmarks and result.right_hand_landmarks:
            # Detectar se está lendo: Cabeça inclinada + mãos próximas ao rosto
            nose_y = result.face_landmarks.landmark[1].y
            hand_y = result.right_hand_landmarks.landmark[8].y  # Ponta do dedo indicador

            if hand_y < nose_y:
                activity = "Lendo"

        elif result.face_landmarks:
            # Detectar se está falando: Boca aberta (diferença entre lábio superior e inferior)
            upper_lip = result.face_landmarks.landmark[13].y
            lower_lip = result.face_landmarks.landmark[14].y

            if abs(upper_lip - lower_lip) > 0.02:
                activity = "Falando"

        elif result.right_hand_landmarks:
            # Detectar se está gesticulando: Mãos levantadas e afastadas do rosto
            wrist_y = result.right_hand_landmarks.landmark[0].y  # Posição do pulso
            nose_y = result.face_landmarks.landmark[1].y if result.face_landmarks else 0.5

            if wrist_y < nose_y:
                activity = "Gesticulando"

        # Armazena a atividade detectada
        activity_counter[activity] += 1

        # Adicionar texto de atividade detectada no vídeo
        cv2.putText(frame, activity, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        # **Detecção de Anomalias**
        if prev_landmarks and result.pose_landmarks:
            current_landmarks = np.array([[lmk.x, lmk.y, lmk.z] for lmk in result.pose_landmarks.landmark])
            prev_landmarks = np.array(prev_landmarks)

            # Calcula a diferença entre frames sucessivos
            motion_diff = np.linalg.norm(current_landmarks - prev_landmarks, axis=1).mean()

            # Define um limiar de movimento anômalo (ajustável)
            anomaly_threshold = 0.05
            if motion_diff > anomaly_threshold:
                anomaly_count += 1  # Conta o número de anomalias detectadas

        prev_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in result.pose_landmarks.landmark] if result.pose_landmarks else None

        # Escrever o quadro no vídeo de saída
        out.write(frame)

        processed_frames += 1

        if processed_frames % 100 == 0:
            print(f"Frames processados: {processed_frames}/{total_frames}")

            end_time = time.time() # Finaliza a contagem do tempo
            total_time = end_time - start_time # Calcula o tempo total
            total_por_frame = total_time/processed_frames

            print(f"Tempo parcial de processamento: {total_time:.2f} segundos") # Imprime o tempo total
            print(f"Tempo parcial de processamento por frame: {total_por_frame:.2f} segundos") # Imprime o tempo total
            print(f"Tempo total de processamento previsto: {total_por_frame*total_frames} segundos") # Imprime o tempo total


        # Exibir o quadro no Google Colab
        cv2_imshow(frame)
        clear_output(wait=True)

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Vídeo processado salvo como {output_path}")

    print("\nResumo do Vídeo:")
    print("-" * 30)

    print(f"Frames totais: {total_frames}")

    # Exibir as atividades predominantes
    print("Atividades Detectadas:")
    for activity, count in activity_counter.most_common():
        print(f"  {activity}: {count} quadros")

    # Exibir as emoções predominantes
    print("\nEmoções Detectadas:")
    for emotion, count in emotion_counter.most_common():
        print(f"  {emotion}: {count} quadros")

    # Exibir o número de anomalias detectadas
    print("\nAnomalias Detectadas:")
    print(f"  {anomaly_count} movimentos anômalos")

    relatorio_path = "results/relatorio.pdf"

    # Create the 'results' directory if it doesn't exist
    os.makedirs(os.path.dirname(relatorio_path), exist_ok=True)

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, "Resumo do Vídeo", ln=True, align='C')
    pdf.cell(200, 10, f"Frames totais: {total_frames}", ln=True, align='L')
    pdf.cell(200, 10, f"Atividades Detectadas:", ln=True, align='L')
    for activity, count in activity_counter.most_common():
      pdf.cell(200, 10, f"{activity}: {count} quadros", ln=True, align='L')
    pdf.cell(200, 10, f"Emoções Detectadas: {emotion} em {count} quadros", ln=True, align='L')
    for emotion, count in emotion_counter.most_common():
      pdf.cell(200, 10, f"{emotion}: {count} quadros", ln=True, align='L')
    pdf.cell(200, 10, f"Anomalias Detectadas: {anomaly_count}", ln=True, align='L')
    pdf.output(relatorio_path)

    print("\nProcessamento finalizado!")


# **Exemplo de uso:**
detect_faces_and_activities(
    "/content/video.mp4",
    "/content/output_video.mp4",
    duration_seconds=3  # Teste rápido com apenas 10 segundos de vídeo
)


Vídeo processado salvo como /content/output_video.mp4

Resumo do Vídeo:
------------------------------
Frames totais: 3326
Atividades Detectadas:
  Em Pe: 90 quadros

Emoções Detectadas:
  fear: 92 quadros
  neutral: 8 quadros
  happy: 7 quadros
  sad: 3 quadros
  angry: 1 quadros

Anomalias Detectadas:
  8 movimentos anômalos

Processamento finalizado!
